# Modeling: Using design parameters

You can read and update parameters that are part of the design.
The simple design in this example has two associated parameters.

## Perform required imports

In [1]:
from pathlib import Path
import requests

from ansys.geometry.core import launch_modeler

The file for this example is in the integration tests folder and can be downloaded.

## Download the example file

Download the file for this example from the integration tests folder in the PyAnsys Geometry repository.

In [2]:
def download_file(url, filename):
    """Download a file from a URL and save it to a local file."""
    response = requests.get(url)
    response.raise_for_status()  # Check if the request was successful
    with open(filename, 'wb') as file:
        file.write(response.content)

# URL of the file to download
url = "https://raw.githubusercontent.com/ansys/pyansys-geometry/main/tests/integration/files/blockswithparameters.dsco"

# Local path to save the file to
file_path = Path.cwd() / "blockswithparameters.dsco"

# Download the file
download_file(url, file_path)
print(f"File is downloaded to {file_path}")

File is downloaded to /home/runner/work/pyansys-geometry/pyansys-geometry/doc/source/examples/03_modeling/blockswithparameters.dsco


## Import a design with parameters

Import the model using the ``open_file()`` method of the modeler.

In [3]:
# Create a modeler object
modeler = launch_modeler()
design = modeler.open_file(file_path)
design.plot()

WARNING -  -  docker_instance - _check_port_availability - Service is already running at port 700...


/home/runner/work/pyansys-geometry/pyansys-geometry/.venv/lib/python3.13/site-packages/ansys/tools/common/cyberchannel.py:188: UserWarning: Starting gRPC client without TLS on localhost:700. This is INSECURE. Consider using a secure connection.
  warn(f"Starting gRPC client without TLS on {target}. This is INSECURE. Consider using a secure connection.")


EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

## Read existing parameters of the design

You can get all the parameters of the design as a list of parameters. Because this
example has two parameters, you see two items in the list.

In [4]:
my_parameters = design.parameters
print(len(my_parameters))

4


A parameter object has a name, value, and unit.

In [5]:
print(my_parameters[0].name)
print(my_parameters[0].dimension_value)
print(my_parameters[0].dimension_type)

print(my_parameters[1].name)
print(my_parameters[1].dimension_value)
print(my_parameters[1].dimension_type)

p1
0.00010872999999999982 meter ** 2
ParameterType.DIMENSIONTYPE_AREA
p2
0.0002552758322160814 meter ** 2
ParameterType.DIMENSIONTYPE_AREA


Parameter values are returned in the default unit for each dimension type. Since default length
unit is meter and default area unit is meter square, the value is returned in square meters.

## Edit a parameter value

You can edit the parameter's name or value by simply setting these fields.
Set the second parameter (p2 value to 350 mm).

In [6]:
parameter1 = my_parameters[1]
parameter1.dimension_value = 0.000440
response = design.set_parameter(parameter1)
print(response)
print(my_parameters[0].dimension_value)
print(my_parameters[1].dimension_value)

ParameterUpdateStatus.SUCCESS
0.00010872999999999982 meter ** 2
0.00044 meter ** 2


After a successful parameter update, the design is updated. If we request the design
plot again, we see the updated design.

In [7]:
design.plot()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

The ``set_parameter()`` method returns a ``Success`` status message if the parameter is updated or
a "FAILURE" status message if the update fails. If the ``p2`` parameter depends on the ``p1``
parameter, updating the ``p1`` parameter might also change the ``p2`` parameter. In such cases,
the method returns ``CONSTRAINED_PARAMETERS``, which indicates other parameters were also updated.

In [8]:
parameter1 = my_parameters[0]
parameter1.dimension_value = 0.000250
response = design.set_parameter(parameter1)
print(response)

ParameterUpdateStatus.CONSTRAINED_PARAMETERS


To get the updated list, query the parameters once again.

In [9]:
my_parameters = design.parameters
print(my_parameters[0].dimension_value)
print(my_parameters[1].dimension_value)

0.00024999999999997263 meter ** 2
0.0005812699999999444 meter ** 2


## Close the modeler

Close the modeler to free up resources and release the connection.

In [10]:
modeler.close()

WARNING - localhost:700 -  client - close - Geometry service was not shut down because it was already running...
